In [8]:
! nvidia-smi

Thu Jun  2 09:18:14 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.32.03    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   61C    P0    29W /  70W |   1684MiB / 15109MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [9]:
!pip install pytorch-tabnet optuna 

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [3]:
import pandas as pd
import numpy as np
from pytorch_tabnet.tab_model import TabNetClassifier
from sklearn.metrics import accuracy_score,classification_report
import optuna as opt
import torch
import os
import joblib

In [4]:
def make_save_cv_model(i,model_name,model,best_params,optim,output_path="./drive/MyDrive/SOLAR_CELL/ML_PROCESSED_DATA/outputs/cross_validated_models"):

    ''' This function saves cross validation model in the corresponding directory ( if the path does not exist it creates the path for it'''


    if os.path.exists(os.path.join(output_path,f"{i}_{model_name}_{optim}")):
        joblib.dump(model, os.path.join(output_path,f"{i}_{model_name}_{optim}/{i}_model.z"))
        with open(os.path.join(output_path,f"{i}_{model_name}_{optim}/model_params.txt"),"w+") as file:
            file.write(str(best_params))
    else:
        os.mkdir(os.path.join(output_path,f"{i}_{model_name}_{optim}"))
        joblib.dump(model, os.path.join(output_path,f"{i}_{model_name}_{optim}/{i}_model.z"))
        with open(os.path.join(output_path,f"{i}_{model_name}_{optim}/model_params.txt"),"w+") as file:
            file.write(str(best_params))

In [5]:
def train(fold_dict,fold,model_name,sc_df,tar_col,optim,optim_trial,k_folds=10,tar_cols="",verbose=1):

    ''' this function is used to train the model with parameters optimization using optuna and cross validation using stratified k_folds'''

    y = sc_df[tar_col]
    x = sc_df.drop([tar_col],axis=1)
    model_name = model_name 
    def objective(trial):
      train_index = fold_dict[fold]["train"]
      test_index = fold_dict[fold]["test"]
      clf = TabNetClassifier(n_d=trial.suggest_int("n_d", 8, 64),
                              n_a =trial.suggest_int("n_a", 8, 64),
                              n_steps = trial.suggest_int("n_steps",3,10),
                              gamma =trial.suggest_float("gamma", 1.0, 2.0),
                              n_independent = trial.suggest_int("n_independent",1,5),
                              n_shared = trial.suggest_int("n_shared",1,5),
                              momentum = trial.suggest_float("momentum", 0.01, 0.4),
                              optimizer_fn = torch.optim.Adam,
                              # scheduler_fn = torch.optim.lr_scheduler,
                              # scheduler_params = {"gamma" :trial.suggest_float("sch-gamma", 0.5, 0.95), "step_size": trial.suggest_int("sch_step_size", 10, 20, 2)},
                              verbose = verbose,
                              device_name = "auto"
                              )
      # print(f" train_index :: {train_index}")
      # print(f" test_index :: {test_index}")
      X_train,X_test = x.iloc[train_index,:], x.iloc[test_index,:]
      # print(X_train.shape, X_test.shape)
      X_train, X_test = X_train.to_numpy(dtype=np.float64), X_test.to_numpy(dtype=np.float64)
      Y_train, Y_test = y.iloc[train_index], y.iloc[test_index]
      Y_train, Y_test = Y_train.to_numpy(dtype=np.float64), Y_test.to_numpy(dtype=np.float64)
      print(Y_train.shape, Y_test.shape)
      clf.fit(X_train, Y_train,
              eval_set=[(X_test, Y_test)],
              eval_metric=['accuracy'])
      Y_pred = clf.predict(X_test)
      print(classification_report(Y_test, Y_pred, labels=[x for x in range(6)]))
      acc = accuracy_score(Y_pred, Y_test)
      return acc

    print(f"Starting optimization for fold : [{fold}/{k_folds}]")
    study = opt.create_study(direction='maximize')
    study.optimize(objective, n_trials=optim_trial)
    best_params = study.best_params
    print(f" Best params for fold : [{fold}/{k_folds}]")
    print(best_params)
    joblib.dump(best_params,f"./drive/MyDrive/SOLAR_CELL/ML_PROCESSED_DATA/outputs/{model_name}/best_params/comp/fold_{fold}_best_params.z")
    with open(f"./drive/MyDrive/SOLAR_CELL/ML_PROCESSED_DATA/outputs/{model_name}/best_params/fold_{fold}_best_params.txt", "w+") as file:file.write(str(best_params))
    print(f"Saved best_params at : outputs/{model_name}/best_params/fold_{fold}_best_params.txt")
    train_index = fold_dict[fold]["train"]
    test_index = fold_dict[fold]["test"]
    X_train,X_test = x.iloc[train_index,:], x.iloc[test_index,:]
    # print(X_train.shape, X_test.shape)
    X_train, X_test = X_train.to_numpy(dtype=np.float64), X_test.to_numpy(dtype=np.float64)
    Y_train, Y_test = y.iloc[train_index], y.iloc[test_index]
    Y_train, Y_test = Y_train.to_numpy(dtype=np.float64), Y_test.to_numpy(dtype=np.float64)
    clf_model = TabNetClassifier(**study.best_params)
    clf_model.fit(X_train,Y_train)
    Y_pred = clf_model.predict(X_test)
    clf_report = classification_report(Y_test, Y_pred, labels=[x for x in range(6)])
    with open(f"./drive/MyDrive/SOLAR_CELL/ML_PROCESSED_DATA/outputs/classification_report/{model_name}_{fold}_classification_report.txt","w+") as file:file.write(str(clf_report))
    accuracy = accuracy_score(Y_pred, Y_test)
    with open(f"./drive/MyDrive/SOLAR_CELL/ML_PROCESSED_DATA/outputs/{model_name}/{model_name}_{fold}_accuracy_score.txt","w+") as file:file.write(f" accuracy :: {str(accuracy)}")
    try:
        print("[++] Saving the model and parameters in corresponding directories")
        make_save_cv_model(fold,model_name,clf_model,best_params,optim=optim)
    except:
        print("[-] Failed to save the model")

In [10]:
use_df = pd.read_csv("./drive/MyDrive/SOLAR_CELL/ML_PROCESSED_DATA/outputs/data/trainable_scaled_balanced.csv")
tar_col = "PCE_categorical"
model_name = "pytorch_tabnet"
optimizer = "Adam"
fold_dict = joblib.load("./drive/MyDrive/SOLAR_CELL/ML_PROCESSED_DATA/inputs/fold_vals/fold_data.z")
fold = 4

In [11]:
train(fold_dict = fold_dict,
      fold = fold,
      model_name=model_name,
      sc_df=use_df,
      tar_col=tar_col,
      optim=optimizer,
      optim_trial = 15)
print(f"[++] Ended the training process for fold {fold}")

[I 2022-06-02 09:18:31,133] A new study created in memory with name: no-name-624541ae-ef44-426a-90ee-96e54a02dfe6


Starting optimization for fold : [4/10]
Device used : cuda
(41688,) (4632,)
epoch 0  | loss: 2.93912 | val_0_accuracy: 0.40501 |  0:00:14s
epoch 1  | loss: 1.36724 | val_0_accuracy: 0.52763 |  0:00:20s
epoch 2  | loss: 1.15798 | val_0_accuracy: 0.58009 |  0:00:28s
epoch 3  | loss: 1.12131 | val_0_accuracy: 0.53886 |  0:00:34s
epoch 4  | loss: 1.10902 | val_0_accuracy: 0.56801 |  0:00:41s
epoch 5  | loss: 0.967   | val_0_accuracy: 0.5041  |  0:00:48s
epoch 6  | loss: 0.90351 | val_0_accuracy: 0.58074 |  0:00:55s
epoch 7  | loss: 0.86171 | val_0_accuracy: 0.54123 |  0:01:01s
epoch 8  | loss: 0.8728  | val_0_accuracy: 0.56412 |  0:01:08s
epoch 9  | loss: 0.86184 | val_0_accuracy: 0.59046 |  0:01:15s
epoch 10 | loss: 0.81461 | val_0_accuracy: 0.58182 |  0:01:22s
epoch 11 | loss: 0.81718 | val_0_accuracy: 0.57254 |  0:01:29s
epoch 12 | loss: 0.79555 | val_0_accuracy: 0.60125 |  0:01:36s
epoch 13 | loss: 0.78265 | val_0_accuracy: 0.60794 |  0:01:43s
epoch 14 | loss: 0.77577 | val_0_accuracy:

[I 2022-06-02 09:25:06,904] Trial 0 finished with value: 0.7031519861830743 and parameters: {'n_d': 23, 'n_a': 48, 'n_steps': 10, 'gamma': 1.6408559766728845, 'n_independent': 2, 'n_shared': 5, 'momentum': 0.07325505148211782}. Best is trial 0 with value: 0.7031519861830743.


              precision    recall  f1-score   support

           0       1.00      1.00      1.00       826
           1       0.45      0.45      0.45       825
           2       0.78      0.76      0.77       822
           3       1.00      1.00      1.00       779
           4       0.41      0.49      0.45       761
           5       0.58      0.47      0.52       619

    accuracy                           0.70      4632
   macro avg       0.70      0.69      0.70      4632
weighted avg       0.71      0.70      0.70      4632

Device used : cuda
(41688,) (4632,)
epoch 0  | loss: 2.2403  | val_0_accuracy: 0.49028 |  0:00:06s
epoch 1  | loss: 1.34148 | val_0_accuracy: 0.27893 |  0:00:12s
epoch 2  | loss: 1.06081 | val_0_accuracy: 0.59801 |  0:00:18s
epoch 3  | loss: 0.87192 | val_0_accuracy: 0.60514 |  0:00:25s
epoch 4  | loss: 0.82713 | val_0_accuracy: 0.63212 |  0:00:31s
epoch 5  | loss: 0.78919 | val_0_accuracy: 0.65566 |  0:00:37s
epoch 6  | loss: 0.77938 | val_0_accuracy: 

[I 2022-06-02 09:30:34,854] Trial 1 finished with value: 0.739853195164076 and parameters: {'n_d': 59, 'n_a': 58, 'n_steps': 8, 'gamma': 1.6013061944605547, 'n_independent': 5, 'n_shared': 3, 'momentum': 0.03907952032477677}. Best is trial 1 with value: 0.739853195164076.


...

              precision    recall  f1-score   support

           0       1.00      1.00      1.00       826
           1       0.61      0.67      0.64       825
           2       0.83      0.94      0.88       822
           3       1.00      1.00      1.00       779
           4       0.54      0.53      0.53       761
           5       0.77      0.54      0.63       619

    accuracy                           0.79      4632
   macro avg       0.79      0.78      0.78      4632
weighted avg       0.79      0.79      0.79      4632

Device used : cuda
(41688,) (4632,)
epoch 0  | loss: 1.28221 | val_0_accuracy: 0.58225 |  0:00:02s
epoch 1  | loss: 0.87143 | val_0_accuracy: 0.6101  |  0:00:04s
epoch 2  | loss: 0.78978 | val_0_accuracy: 0.64076 |  0:00:06s
epoch 3  | loss: 0.74831 | val_0_accuracy: 0.66408 |  0:00:09s
epoch 4  | loss: 0.72258 | val_0_accuracy: 0.67012 |  0:00:11s
epoch 5  | loss: 0.71251 | val_0_accuracy: 0.67314 |  0:00:13s
epoch 6  | loss: 0.69924 | val_0_accuracy: 

[I 2022-06-02 10:22:28,545] Trial 13 finished with value: 0.8052677029360967 and parameters: {'n_d': 8, 'n_a': 64, 'n_steps': 3, 'gamma': 1.475493044135972, 'n_independent': 4, 'n_shared': 2, 'momentum': 0.17448775099959518}. Best is trial 13 with value: 0.8052677029360967.


              precision    recall  f1-score   support

           0       1.00      1.00      1.00       826
           1       0.68      0.64      0.66       825
           2       0.85      0.94      0.89       822
           3       1.00      1.00      1.00       779
           4       0.56      0.55      0.56       761
           5       0.67      0.67      0.67       619

    accuracy                           0.81      4632
   macro avg       0.80      0.80      0.80      4632
weighted avg       0.80      0.81      0.80      4632

Device used : cuda
(41688,) (4632,)
epoch 0  | loss: 1.55447 | val_0_accuracy: 0.53713 |  0:00:03s
epoch 1  | loss: 1.07848 | val_0_accuracy: 0.56563 |  0:00:06s
epoch 2  | loss: 1.01044 | val_0_accuracy: 0.58225 |  0:00:09s
epoch 3  | loss: 0.91612 | val_0_accuracy: 0.6237  |  0:00:13s
epoch 4  | loss: 0.84044 | val_0_accuracy: 0.62802 |  0:00:16s
epoch 5  | loss: 0.79967 | val_0_accuracy: 0.64011 |  0:00:19s
epoch 6  | loss: 0.77585 | val_0_accuracy: 

[I 2022-06-02 10:28:01,352] Trial 14 finished with value: 0.7856217616580311 and parameters: {'n_d': 29, 'n_a': 19, 'n_steps': 5, 'gamma': 1.5124204349864008, 'n_independent': 4, 'n_shared': 2, 'momentum': 0.17088385235492723}. Best is trial 13 with value: 0.8052677029360967.


              precision    recall  f1-score   support

           0       1.00      1.00      1.00       826
           1       0.62      0.63      0.62       825
           2       0.80      0.95      0.87       822
           3       1.00      1.00      1.00       779
           4       0.54      0.53      0.54       761
           5       0.72      0.53      0.61       619

    accuracy                           0.79      4632
   macro avg       0.78      0.77      0.77      4632
weighted avg       0.78      0.79      0.78      4632

 Best params for fold : [4/10]
{'n_d': 8, 'n_a': 64, 'n_steps': 3, 'gamma': 1.475493044135972, 'n_independent': 4, 'n_shared': 2, 'momentum': 0.17448775099959518}
Saved best_params at : outputs/pytorch_tabnet/best_params/fold_4_best_params.txt
Device used : cuda
No early stopping will be performed, last training weights will be used.
epoch 0  | loss: 1.28221 |  0:00:02s
epoch 1  | loss: 0.88175 |  0:00:04s
epoch 2  | loss: 0.79022 |  0:00:06s
epoch 3  |

Fold 0 has started running on 20-05-22 


Fold 0 has completed sucessfully on 17:00 20-05-22

Fold 1 has started running at 15:15 21-05-22

Fold 2 has started running at 09:45 22-05-22

Fold 2 has completed sucessfully on 10:58 22-05-22

Fold 3 has started running at 18:40 22-05-22

Fold 3 has completed sucessfully on 22-05-22

Fold 4 completed sucessfully on 21:04 on 22-05-22

Fold 5 started at 18:21 on 23-05-22

Fold 5 completed sucessfully on 19:44 on 23-05-22

Fold 6 started at 12:53 on 24-05-22

Fold 6 has completed at 14:14 on 24-05-22

Fold 7 started at 14:18 on 24-05-22

Fold 7 execution failed due to colab gpu time limit

Fold 7 trial 1 started at 11:00 on 25-05-22

Fold 7 has completed sucessfully at 12:14 on 25-05-22 

Fold 8 has started at 9:38 on 26-05-22

Fold 8 filed due to interrupted internet connection

Fold 8 trial 1 started at 13:38 on 26-05-22

Fold 8 has successfully executed at 15:33 on 26-05-22

Fold 9 has started at 13:35 on 27-05-22

Fold 9 has completed at 14:55 on 27-05-22

Editing with rectified dataset witout duplicacy because of space values

Fold 0 started at 13:21 on 28-05-22

Fold 0 completed sucessfully at 14:46 on 28-05-22

Fold 1 Failed to run due to some index error

_____ Restarting the training process again due to data distribution failure____



Fold 0 started at 10:47 on 30-05-22

Fold 0 completed successfully at 12:30 on 30-05-22

Fold 1 started at 8:27 on 31-05-22

Fold 1 execution failed due to runtime disconnection

Fold 1 started again at 9:38 on 31-05-22

Fold 1 execution failed due to gpu disconnect 

Fold 1 started again at 8:36 on 1-06-22

Fold 1 execution failed due to network disconnection

Fold 1 started again at 13:11 on 01-06-22

Fold 1 has succesfully executed at 14:25 on 01-06-22

Fold 2 started at 14:29 on 01-06-22

Fold 2 completed succesfully at 16:00 on 01-06-22

Fold 3 started at 09:43 on 02-06-22

Fold 3 execution failed due to gpu server disconnection 

Fold 3 started again at 13:34 on 02-06-22

Fold 3 ran successfully at 14:47 on 02-06-22

Fold 4 started at 14:48 on 02-06-22

Fold 4 ran successfully at 04:01 on 02-06-22